In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [2]:
training_data=pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")

In [3]:
testing_data=pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")

In [4]:
training_data.shape[0]

12120

In [5]:
testing_data.shape[0]

5195

In [6]:
training_data.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [7]:
!pip install transformers

Define TPU **usage**

In [8]:
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, XLMRobertaModel, XLMRobertaForSequenceClassification

In [9]:
import tensorflow as tf
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(TPU)
except ValueError:
    tpu_strategy = tf.distribute.get_strategy() # for CPU and single GPU
print('Number of replicas:', tpu_strategy.num_replicas_in_sync)

Number of replicas: 1


In [10]:
device = 'cpu'
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'using {device} : {torch.cuda.get_device_name()}\n')
else:
    print(f'using {device}\n')

using cuda : Tesla P100-PCIE-16GB



In [11]:
MAX_LENGTH = 256
BATCH_SIZE = 16
MODEL_NAME='xlm-roberta-base'

TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [12]:
class TwoSentencesDataset(Dataset):
    def __init__(self, sentence_1_list, sentence_2_list, labels = None):
        self.sentence_1_list = sentence_1_list
        self.sentence_2_list = sentence_2_list
        self.labels = labels

    def __len__(self):
        return len(self.sentence_1_list)

    def __getitem__(self, idx):
        
        token_dict = TOKENIZER(
            self.sentence_1_list[idx],self.sentence_2_list[idx],
            padding='max_length',
            max_length=MAX_LENGTH, 
            truncation=True,
            return_tensors = 'pt'
        )
        
        if self.labels == None:
            return (token_dict.input_ids[0], token_dict.attention_mask[0])
        else:
            target = torch.tensor(self.labels[idx])
            return (token_dict.input_ids[0], token_dict.attention_mask[0], self.labels[idx])

train_dataset = TwoSentencesDataset(
                    training_data['premise'].to_list(), 
                    training_data['hypothesis'].to_list(), 
                    training_data['label'].to_list())
test_dataset = TwoSentencesDataset(
                    testing_data['premise'].to_list(), 
                    testing_data['hypothesis'].to_list(), 
                    labels = None)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
from transformers import TFAutoModel

with tpu_strategy.scope():
  model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME,num_labels = 3, output_attentions = False,output_hidden_states = False).to(device)
  optimizer = optim.Adam(model.parameters(), lr=1e-5)
  criterion = nn.CrossEntropyLoss()

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Define XLMRoberta model, optimizer and loss function


In [14]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train_loss = 0
    output_labels_in_epoch, actual_labels_in_epoch = np.array([]), np.array([])
    pbar = tqdm(enumerate(train_dataloader), total = len(train_dataloader))
    for i, (input_ids, attention_mask, labels) in pbar:
        
        # put tensors into devices and forward them
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        output = output.logits
        
        # calculate loss and add to overall loss of this epoch
        loss = criterion(output,labels)
        train_loss += loss.item()
        
        # predicted labels and actual labels and add to overall list of this epoch
        output_labels = torch.argmax(output,dim=1).cpu().detach().numpy()
        actual_labels = labels.cpu().detach().numpy()
        output_labels_in_epoch = np.append(output_labels_in_epoch, output_labels)
        actual_labels_in_epoch = np.append(actual_labels_in_epoch, actual_labels)
        acc = np.sum(output_labels_in_epoch == actual_labels_in_epoch) / len(output_labels_in_epoch)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        train_desc = f'Epoch [{epoch}/{EPOCHS}], Loss: {train_loss/(i+1):.4f}, Acc: {acc:.4f}'
        pbar.set_description(desc = train_desc)

Epoch [9/10], Loss: 0.0374, Acc: 0.9889: 100%|██████████| 758/758 [05:11<00:00,  2.44it/s]


Prediction process

In [15]:
result_labels = np.array([])
with torch.no_grad():
    for i, (input_ids, attention_mask) in tqdm(enumerate(test_dataloader)):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        output_label = torch.argmax(output.logits,dim=1).cpu().detach().numpy()
        result_labels = np.append(result_labels,output_label)

325it [00:42,  7.60it/s]


In [16]:
testing_data['prediction'] = result_labels
testing_data['prediction'] = testing_data['prediction'].astype('int')
testing_data[['id','prediction']].to_csv('submission.csv',index=False)